In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
from implementations import *
from Data_cleaning import *
from helpers import *

In [2]:
x_train, x_test, y_train, train_ids, test_ids = load_csv_data("data\dataset\dataset")

In [13]:
#Turn y into 0s and 1s
y_tr_ = (y_train + 1) / 2
y_tr = y_tr_[:180000]
y_va = y_tr_[180000:240000]
y_te = y_tr_[240000:300000]
print(y_tr.shape, y_te.shape, y_tr, y_te)

(180000,) (60000,) [0. 0. 0. ... 0. 0. 0.] [0. 0. 0. ... 0. 0. 0.]


In [26]:

def remove_categorical_features(X, threshold=100):
    """
    Removes categorical (low-cardinality) features from a numeric dataset.
    A feature is dropped if it has fewer than `threshold` unique values.
    """
    X_np = np.asarray(X)
    n_samples, n_features = X_np.shape
    keep_mask = np.array([
        np.unique(X_np[:, j]).size >= threshold for j in range(n_features)
    ])
    X_num = X_np[:, keep_mask]
    return X_num, keep_mask

def anova_f_test(X, y):
    """
    Compute ANOVA F-statistic for each feature using NumPy only.
    
    Parameters
    ----------
    X : np.ndarray, shape (n_samples, n_features)
        Feature matrix.
    y : np.ndarray, shape (n_samples,)
        Class labels.
    
    Returns
    -------
    F_values : np.ndarray, shape (n_features,)
        F-statistics for each feature.
    """
    n_samples, n_features = X.shape
    classes = np.unique(y)
    k = len(classes)
    
    overall_means = np.nanmean(X, axis=0)  # handle NaNs safely
    
    # Initialize sums of squares
    ssb = np.zeros(n_features)
    ssw = np.zeros(n_features)
    
    for c in classes:
        X_c = X[y == c]
        n_c = X_c.shape[0]
        mean_c = np.nanmean(X_c, axis=0)
        
        ssb += n_c * (mean_c - overall_means) ** 2
        ssw += np.nansum((X_c - mean_c) ** 2, axis=0)
    
    # Avoid division by zero
    ssw = np.where(ssw == 0, np.nan, ssw)
    
    F = (ssb / (k - 1)) / (ssw / (n_samples - k))
    
    return F
def logistic_regression_penalized_gradient_descent_weighted(y, X, lambda_, gamma=0.5, max_iter=10000, threshold=1e-8):
    """
    Logistic regression with L2 regularization and class imbalance weighting.
    """
    n, d = X.shape
    tx = np.c_[np.ones((n, 1)), X]
    w = np.zeros(d + 1)

    # Compute class weights
    n_pos = np.sum(y == 1)
    n_neg = np.sum(y == 0)
    w_pos = n / (2 * n_pos)
    w_neg = n / (2 * n_neg)

    for it in range(max_iter):
        y_pred = 1 / (1 + np.exp(-tx @ w))

        # Weighted gradient
        weights = np.where(y == 1, w_pos, w_neg)
        error = y_pred - y
        grad = (tx.T @ (weights * error)) / n + lambda_ * np.r_[0, w[1:]]  # no reg. on bias

        # Update weights
        w -= gamma * grad

        # Convergence check
        if np.linalg.norm(grad) < threshold:
            break

    return w



In [27]:
# --- CLEANING ---
# 0. Remove categorical (low-cardinality) features
x_num, cat_mask = remove_categorical_features(x_train, threshold=1000)
print(x_num.shape)
# 1. Remove features with too many NaNs
x_clean, keep_mask = remove_nan_features(x_num)

x_clean_tr = x_clean[:180000]
x_clean_va = x_clean[180000:240000]
x_clean_te = x_clean[240000:300000]

# 2. Impute remaining missing values
# Compute feature means from the training data (ignore NaNs)
train_means = np.nanmean(x_clean_tr, axis=0)

# Replace NaNs in training data with training means
inds_tr = np.where(np.isnan(x_clean_tr))
x_clean_tr[inds_tr] = np.take(train_means, inds_tr[1])

# Replace NaNs in validation data with training means
inds_va = np.where(np.isnan(x_clean_va))
x_clean_va[inds_va] = np.take(train_means, inds_va[1])

# Replace NaNs in test data with training means
inds_te = np.where(np.isnan(x_clean_te))
x_clean_te[inds_te] = np.take(train_means, inds_te[1])

# Compute F-scores using only the training data
F_scores = anova_f_test(x_clean_tr, y_tr)

# Select top 30 features
top_k = 30
top_features_idx = np.argsort(F_scores)[-top_k:][::-1]

# Apply the same feature selection to all sets
x_anova_tr = x_clean_tr[:, top_features_idx]
x_anova_va = x_clean_va[:, top_features_idx]
x_anova_te = x_clean_te[:, top_features_idx]

# --- STANDARDIZATION ---
x_tr_st, means, stds = standardize_features(x_anova_tr)
x_va_st = (x_anova_va - means) / stds
x_te_st = (x_anova_te - means) / stds


print(x_tr_st.shape)


(328135, 11)
(180000, 8)


In [24]:
#check shapes
print(x_tr_st.shape, y_tr.shape)

(180000, 27) (180000,)


In [25]:
# ============================================================
# CROSS-VALIDATION PIPELINE FOR LOGISTIC REGRESSION (RIDGE)
# ============================================================
# ------------------------------------------------------------
# 1. Metric function
# ------------------------------------------------------------
def f1_score(y_true, y_pred):
    """Compute F1 score between true and predicted labels."""
    tp = np.sum((y_true == 1) & (y_pred == 1))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    fn = np.sum((y_true == 1) & (y_pred == 0))
    if tp + fp == 0 or tp + fn == 0:
        return 0.0
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    if precision + recall == 0:
        return 0.0
    return 2 * precision * recall / (precision + recall)


# ------------------------------------------------------------
# 2. Stratified K-Fold Split
# ------------------------------------------------------------
def stratified_kfold_indices(y, k=5, seed=42):
    """Return list of (train_idx, val_idx) preserving class ratio."""
    np.random.seed(seed)
    idx_pos = np.where(y == 1)[0]
    idx_neg = np.where(y == 0)[0]
    np.random.shuffle(idx_pos)
    np.random.shuffle(idx_neg)
    pos_folds = np.array_split(idx_pos, k)
    neg_folds = np.array_split(idx_neg, k)

    folds = []
    for i in range(k):
        val_idx = np.concatenate([pos_folds[i], neg_folds[i]])
        train_idx = np.setdiff1d(np.arange(len(y)), val_idx)
        folds.append((train_idx, val_idx))
    return folds


# ------------------------------------------------------------
# 3. Cross-validation for λ and α search
# ------------------------------------------------------------
def cross_validate_lambda_alpha(
    X, y,
    lambdas, alphas,
    k=5,
    gamma=0.5, max_iter=10000, threshold=1e-8,
    seed=42
):
    """
    Perform k-fold cross-validation to find best (lambda, alpha)
    for penalized logistic regression based on mean F1 score.

    Uses your existing logistic_regression_penalized_gradient_descent_demo().
    """

    best_lambda, best_alpha, best_mean_f1 = None, None, -1
    folds = stratified_kfold_indices(y, k=k, seed=seed)

    # Store all mean F1s (optional, for visualization)
    f1_matrix = np.zeros((len(lambdas), len(alphas)))

    for i_lam, lam in enumerate(lambdas):
        for i_alpha, alpha in enumerate(alphas):
            fold_f1s = []

            # ----- Perform k-fold cross-validation -----
            for i, (train_idx, val_idx) in enumerate(folds):
                X_train, y_train = X[train_idx], y[train_idx]
                X_val, y_val = X[val_idx], y[val_idx]

                # Train penalized logistic regression with current lambda
                w = logistic_regression_penalized_gradient_descent_weighted(
                y_train, X_train,
                lambda_=lam,
                gamma=gamma,
                max_iter=max_iter,
                threshold=threshold
)


                # Predict probabilities on validation data
                tx_val = np.c_[np.ones((y_val.shape[0], 1)), X_val]
                y_proba = sigmoid(tx_val @ w)

                # Apply classification threshold α
                y_pred = (y_proba >= alpha).astype(int)

                # Compute F1 on this fold
                fold_f1s.append(f1_score(y_val, y_pred))

            # ----- Average F1 across folds -----
            mean_f1 = np.mean(fold_f1s)
            f1_matrix[i_lam, i_alpha] = mean_f1

            # Track best parameters
            if mean_f1 > best_mean_f1:
                best_mean_f1 = mean_f1
                best_lambda = lam
                best_alpha = alpha

    print(f"Best λ = {best_lambda}, Best α = {best_alpha}, Mean F1 = {best_mean_f1:.4f}")
    return best_lambda, best_alpha, best_mean_f1, f1_matrix


In [18]:
print("Any NaN in X?", np.isnan(x_tr_st).any())
print("Any NaN in y?", np.isnan(y_tr).any())
print("Unique values in y:", np.unique(y_tr))
print("Shape X:", x_tr_st.shape)
print("Shape y:", y_tr.shape)


Any NaN in X? False
Any NaN in y? False
Unique values in y: [0. 1.]
Shape X: (180000, 30)
Shape y: (180000,)


In [20]:
lambdas = [1e-4, 1e-3, 1e-2, 1e-1, 1]
alphas  = np.linspace(0.3, 0.5, 3)

best_lambda, best_alpha, best_f1 = None, None, -1

for lam in lambdas:
    # Train on TRAIN set
    w = logistic_regression_penalized_gradient_descent_weighted(
        y_tr, x_tr_st,
        lambda_=lam,
        gamma=0.5,
        max_iter=500
    )

    # Predict probabilities on VALIDATION set
    tx_va = np.c_[np.ones((x_va_st.shape[0], 1)), x_va_st]
    y_val_proba = sigmoid(tx_va @ w)

    # Search over classification thresholds
    for alpha in alphas:
        y_val_pred = (y_val_proba >= alpha).astype(int)
        f1 = f1_score(y_va, y_val_pred)

        if f1 > best_f1:
            best_f1 = f1
            best_lambda = lam
            best_alpha = alpha

print(f"Best λ = {best_lambda}, Best α = {best_alpha}, Val F1 = {best_f1:.4f}")

Best λ = 0.001, Best α = 0.5, Val F1 = 0.2933


In [21]:
# Combine TRAIN + VALIDATION data
X_all = np.vstack([x_tr_st, x_va_st])
y_all = np.concatenate([y_tr, y_va])

# Retrain with best λ
final_w = logistic_regression_penalized_gradient_descent_weighted(
    y_all, X_all,
    lambda_=best_lambda,
    gamma=0.5,
    max_iter=5000
)


In [22]:
tx_te = np.c_[np.ones((x_te_st.shape[0], 1)), x_te_st]
y_te_proba = sigmoid(tx_te @ final_w)
y_te_pred  = (y_te_proba >= best_alpha).astype(int)

final_f1 = f1_score(y_te, y_te_pred)
final_acc = np.mean(y_te_pred == y_te)

print(f"Test F1 = {final_f1:.4f}, Test Accuracy = {final_acc:.4f}")


Test F1 = 0.2925, Test Accuracy = 0.6696


In [ ]:
def predict_labels_logistic(x, w, threshold=0.5, return_original_labels=True):

    # add bias term
    tx = np.c_[np.ones(x.shape[0]), x]
    
    # compute probabilities
    probs = sigmoid(tx @ w)
    
    # threshold
    preds = (probs >= threshold).astype(int)
    
    # convert to {-1, 1} if desired
    if return_original_labels:
        preds = 2 * preds - 1
    
    return preds